Imports

In [2]:
import pandas as pd
import random
import numpy as np
import warnings

Global functions

In [3]:
def get_accident_df(subset="standard_small", n_rows=10000, is_refined=False):
    warnings.warn("This function is outdated and should not be used.")
    if is_refined:
        df = get_accident_df(subset=subset, n_rows=n_rows, is_refined=False)
        return refine_df(df)

    if subset == "standard_small":
        return pd.read_csv("DataSubsetRaw.csv")
    if subset == "new":
        n = 3513617 #number of rows in the dataset
        s = n_rows #number of rows to sample
        skip = sorted(random.sample(range(1, n+1), n-s)) #rows to be skipped
        return pd.read_csv("US_Accidents_June20.csv", skiprows=skip)
    if subset == "all":
        return pd.read_csv("US_Accidents_June20.csv")

Internal functions

In [ ]:
def get_small_df(size=100):
    n = 3513617 #number of rows in the dataset                          
    s = size #number of rows to sample                                
    skip = sorted(random.sample(range(1, n+1), n-s)) #rows to be skipped
    return pd.read_csv("US_Accidents_June20.csv", skiprows=skip)        

In [4]:
def refine_df(df):
    df = change_column_names(df)
    df = convert_to_metric(df)
    df = timeStamp_df(df)
    df = duration_df(df)
    return df

In [5]:
def change_column_names(df):
    col_names = df.columns
    col_names = [name.replace("_", "") for name in col_names]
    col_names = [name[:loc] if (loc := name.find("(")) != -1 else name for name in col_names]
    df.columns = col_names
    return df

In [6]:
def convert_to_metric(df):
    #miles to km
    df["Distance"] = df["Distance"] * 1.60934

    #F to C
    df["Temperature"] = (df["Temperature"] - 32) * 5 / 9

    #F to C
    df["WindChill"] = (df["WindChill"] - 32) * 5 / 9

    #inches of mercury to bar
    df["Pressure"] = df["Pressure"] / 29.53

    #miles to km
    df["Visibility"] = df["Visibility"] * 1.60934

    #mph to kph
    df["WindSpeed"] = df["WindSpeed"] * 1.60934

    #inches to cm
    df["Precipitation"] = df["Precipitation"] * 2.54
    return df

In [7]:
#This function converts Start_Time and End_Time data in dataframe to 'datetime64[ns]' datatype.
def timeStamp_df(df):
    df['StartTime'] = pd.to_datetime(df['StartTime'])
    df['EndTime'] = pd.to_datetime(df['EndTime'])
    return df

In [8]:
#This function creates column in df with duration data as dataype 'timedelta64[ns]'.
def duration_df(df):
    df['Duration'] = (df['EndTime']-df['StartTime'])
    return df

Testing

In [19]:
df = pd.read_csv("US_Accidents_June20.csv").sample(200000)
df = refine_df(df)

In [20]:
df_accident = df
df_city_pop = pd.read_csv("CityPopulations.csv")

In [ ]:
df_accident_count = df_accident.groupby(["City", "State"]).count().reset_index()


In [22]:
df_accident_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9033 entries, 0 to 9032
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   City                  9033 non-null   object
 1   State                 9033 non-null   object
 2   ID                    9033 non-null   int64 
 3   Source                9033 non-null   int64 
 4   TMC                   9033 non-null   int64 
 5   Severity              9033 non-null   int64 
 6   StartTime             9033 non-null   int64 
 7   EndTime               9033 non-null   int64 
 8   StartLat              9033 non-null   int64 
 9   StartLng              9033 non-null   int64 
 10  EndLat                9033 non-null   int64 
 11  EndLng                9033 non-null   int64 
 12  Distance              9033 non-null   int64 
 13  Description           9033 non-null   int64 
 14  Number                9033 non-null   int64 
 15  Street                9033 non-null   

In [30]:
df_accident_count = df_accident_count.loc[:, ["City", "State", "ID"]]

In [32]:
df_accident_count["Amount"] = df_accident_count["ID"]

In [34]:
df_city_pop.head()

,City,State,Population,lat,lon
0,Marysville,Washington,63269,48.051764,-122.177082
1,Perris,California,72326,33.782519,-117.228648
2,Cleveland,Ohio,390113,41.499320,-81.694361
3,Worcester,Massachusetts,182544,42.262593,-71.802293
4,Columbia,South Carolina,133358,34.000710,-81.034814
